In [ ]:
import functools
import geopy
from imposm.parser import OSMParser
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyproj
import requests
import scipy as sp
import rtree
import seaborn as sb
from scipy import signal
# import shapely
import shapely.geometry
%pylab inline

import data_munging

In [ ]:
# simple class that handles the parsed OSM data.
class HighwayCounter(object):
    highways = 0

    def ways(self, ways):
        # callback method for ways
        for osmid, tags, refs in ways:
            if 'highway' in tags:
              self.highways += 1

class SegmentFinder(object):
    segments = []
    
    def ways(self, ways):
        # callback method for ways
        for osmid, tags, refs in ways:
            prev_ref = None
            for ref in refs:
                if prev_ref is not None:
                    self.segments.append((prev_ref, ref))
                prev_ref = ref

class NodeCoords(object):
    node_coords = dict()

    def nodes(self, nodes):
        # callback method for nodes
        for osmid, tags, coords in nodes:
            self.node_coords[osmid] = coords

# instantiate counter and parser and start parsing
all_segments = SegmentFinder()
some_coords = NodeCoords()
p = OSMParser(concurrency=4,
              nodes_callback=some_coords.nodes,
              ways_callback=all_segments.ways)
p.parse('../../osrm/osm_data/chicago_illinois.osm.pbf')
# done

In [ ]:
segments_idx = rtree.index.Index()
total_failures = 0
for segment in SegmentFinder.segments:
    try:
        lat0, lon0 = some_coords.node_coords[segment[0]]
        lat1, lon1 = some_coords.node_coords[segment[1]]
        coord0 = data_munging.NAD83(lon0, lat0)
        coord1 = data_munging.NAD83(lon1, lat1)
        segments_idx.insert(id=segment, coordinates=data_munging.coords_to_bb(coord0, coord1),
                           obj=(coord0, coord1))
    except:
        total_failures += 1
    break

In [ ]:
print total_failures

In [ ]:
rides, readings = data_munging.read_raw_data()
readings = data_munging.clean_readings(readings)
readings = data_munging.add_proj_to_readings(readings, data_munging.NAD83)

In [ ]:
start_coords = readings[['start_lat', 'start_lon']]
end_coords = readings[['end_lat', 'end_lon']]

In [ ]:
bb_areas = list()
bb_width = list()
bb_height = list()
bb_max = list()
for i, reading in readings.iterrows():
    the_bb = data_munging.reading_to_bb(reading)
    bb_area = data_munging.area_of_bb(the_bb)
    bb_areas.append(bb_area)
    bb_width.append(np.abs(reading.start_x - reading.end_x))
    bb_height.append(np.abs(reading.start_y- reading.end_y))    
    bb_max.append(np.max([bb_width[-1], bb_height[-1]]))
#     intx_segments = segments_idx.intersection()
#     while len(intx_segments) == 0:        

In [ ]:
plt.hist(bb_areas)
plt.plot()

In [ ]:
plt.scatter(bb_width, bb_height)
plt.show()

In [ ]:
plt.hist(bb_max)
plt.show()

In [ ]:
n,p = readings.shape

In [ ]:
total_bb = pd.DataFrame(0, index=range(10), columns=[0.1, 0.4])

In [ ]:
total_bb = pd.DataFrame(0, index=range(n), columns=[0.5 ** x for x in range(10)])

In [ ]:
for i, reading in readings.iterrows():
    the_bb = data_munging.reading_to_bb(reading)
    expanded_bb = data_munging.expand_bb(the_bb, 3.0)
    closest_segment = None
    closest_dist = np.inf
    for segment in segments_idx.intersection(expanded_bb):
        print segment
        break
#        if shapely.geometry.LineString([(0,0), (1,1)]).distance(shapely.geometry.Point((0.5, 0.4)))

In [ ]:
for i in total_bb:
    for col in total_bb:
        the_bb = data_munging.reading_to_bb(readings.ix[i, :], 1)
        expand_bb = data_munging.expand_bb(readings.ix[i, :])
        total_bb[i, col] = len(segments_idx.intersection(data_munging.reading_to_bb(readings.ix[i,:])))

In [ ]:
print len(all_segments.segments)
print len(some_coords.node_coords)

In [ ]:
rides, readings = data_munging.read_raw_data()
readings = data_munging.clean_readings(readings)
readings = data_munging.add_proj_to_readings(readings, data_munging.NAD83)

In [ ]:
readings.columns

In [ ]:
readings.loc[0, ['num_accel_x', 'num_accel_y', 'num_accel_z', 'num_accel_total']]


In [ ]:
print rides.shape
print readings.shape
n, p = readings.shape

In [ ]:
nearest_request = 'http://127.0.0.1:5000/nearest?loc={0},{1}'
map_request = 'http://127.0.0.1:5000/match?loc={0},{1}&t={2}&loc={3},{4}&t={5}'

In [ ]:
map_request

In [ ]:
for snapped_var in ['snapped_' + var for var in ['start_lat', 'start_lon', 'end_lat', 'end_lon']]:
    readings[snapped_var] = 0

In [ ]:
osrm_response['matchings'][0]['matched_points']

In [ ]:
total_calls = 0
for i, reading in readings.iterrows():
    data = tuple(reading[['start_lat', 'start_lon']])+9
    print i
    print data 
    osrm_request = nearest_request.format(*data)
    osrm_response = requests.get(osrm_request).json()
    print osrm_response
    total_calls += 1
    if total_calls > 100:
        break

In [ ]:
total_calls = 0
for i, reading in readings.iterrows():
    data = tuple(reading[['start_lat', 'start_lon']]) + (0,) + tuple(reading[['end_lat', 'end_lon']]) + (1,)   
    print i
    print data 
    osrm_request = map_request.format(*data)
    osrm_response = requests.get(osrm_request).json()
    print osrm_response
    total_calls += 1
    if total_calls > 100:
        break

In [ ]:
for i, reading in readings.iterrows():
    data = tuple(reading[['start_lat', 'start_lon']]) + (0,) + tuple(reading[['end_lat', 'end_lon']]) + (1,)   
    osrm_request = map_request.format(*data)
    osrm_response = requests.get(osrm_request).json()
    print osrm_response
    readings.loc[i, ['snapped_start_lat', 'snapped_start_lon']] = osrm_response['matchings'][0]['matched_points'][0]
    break

In [ ]:
osrm_request = map_request.format(*reading[['start_lat', 'start_lon']], 0)


In [ ]:
(0, 4) + (0,5,9)

In [ ]:
# This does not work because OSRM does not accept floats as times. 
test_map_request = map_request.format(*tuple(a_reading[['start_lat', 'start_lon', 'start_time',
                                                'end_lat', 'end_lon', 'end_time']]))

In [ ]:
test_map_request = map_request.format(a_reading['start_lat'],
                                      a_reading['start_lon'], 
                                      0,
                                      a_reading['end_lat'],
                                      a_reading['end_lon'],
                                      1)

In [ ]:
print test_map_request
print requests.get(test_map_request).json()